In [28]:
import pandas as pd
from datetime import datetime
import numpy as np

test_data = pd.read_csv('D:/Valerian/Documents/OneDrive/Python/ДопОбр Анализ данных/Практика Нетодология/sales_prediction/Команда_11/test.csv', encoding='iso-8859-1')

train_data = pd.read_csv('D:/Valerian/Documents/OneDrive/Python/ДопОбр Анализ данных/Практика Нетодология/sales_prediction/Команда_11/train.csv', parse_dates=['Date'], low_memory=False)

# Загрузка данных из файла store.csv
store_data = pd.read_csv('D:/Valerian/Documents/OneDrive/Python/ДопОбр Анализ данных/Практика Нетодология/sales_prediction/Команда_11/store.csv', encoding='iso-8859-1')




In [24]:
df_form = train_data.copy()

df_form.replace({' - ': np.nan, '\\N': np.nan, 'NaN': np.nan}, inplace=True)
df_form = df_form.dropna()

print(df_form.head())

   Store  DayOfWeek       Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5 2015-07-31   5263        555     1      1            0   
1      2          5 2015-07-31   6064        625     1      1            0   
2      3          5 2015-07-31   8314        821     1      1            0   
3      4          5 2015-07-31  13995       1498     1      1            0   
4      5          5 2015-07-31   4822        559     1      1            0   

   SchoolHoliday  
0              1  
1              1  
2              1  
3              1  
4              1  


In [31]:
print(train_data['Date'].dtype)

datetime64[ns]
